## TODO

- add screwholes to plate

In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

from plate import Config, Shape, get_key_positions, get_key_hole_shape, get_keys

from functools import partial
import math

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


In [67]:


nrows = 4
rkn = [7,7,7,6]
switchholesize=13.9
cc=Config(nrows, rkn, shape=Shape.LEAN, cs=switchholesize+ 3.4505, rs= switchholesize+5.508, switchHoleSize=switchholesize)
key_hole_shape = get_key_hole_shape(cc)

kp = get_key_positions(cc)
get_keys(kp, key_hole_shape, cc)

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [3]:
def get_base(config: Config, kp, thickness, window=False):

    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                      config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    base = cq.Sketch()
   

    base = base.push(kp.values())
    if config.shape == Shape.LEAN:
        base = base.rect(foot_x, foot_y)\
            .faces().clean().vertices().fillet(2.5).faces()\
            .wires().offset(0).clean()
    elif config.shape == Shape.HULL:
        base = base.rect(foot_x, foot_y)\
            .faces().hull().clean().wires().offset(12)

    base = cq.Workplane().placeSketch(base).extrude(thickness)


    if window:
        win = cq.Sketch().push(kp.values()).rect(config.columnSpacing / 2 +
                                                 config.switchHoleSize, config.rowSpacing / 2 + config.switchHoleSize)
        win = win.clean().faces().vertices().fillet(1)
        win = cq.Workplane().placeSketch(win).extrude(thickness).rotate((0, 0, 0), (0, 0, 1), config.angle).translate(
            (config.hOffset, 0))
        #if not config.split:
        win = win.mirror('YZ', union=True)
        base = base.cut(win)

    return base





In [101]:
nrows = 5
rkn = [1,3,5,2,3]

cc=Config(nrows, rkn, shape=Shape.LEAN)
kp = get_key_positions(cc)
plate = get_base(cc, kp, thickness=2, window=False).cut(get_keys(kp, key_hole_shape, cc))
plate

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [95]:

def rotate(config: Config, pt):
    ang = math.radians(config.angle)
    x, y = pt
    return (x * math.cos(ang) - y * math.sin(ang) + config.hOffset,
            x * math.sin(ang) + y * math.cos(ang))




In [100]:
def get_screw_positions(config: Config) -> [(float, float)]:
    sp=[]
    x_centre = max(config.row_key_num)* config.rowSpacing*0.5
    row_holes = list(reversed( [config.row_key_num[0]]+ config.row_key_num)) 
    for col_num in range(0, cc.nrows+1,2):    
        row_size = row_holes[col_num]
        centre = row_size* config.rowSpacing*0.5

        if centre != x_centre:
            x_trans = x_centre - centre
        else:
            x_trans = 0
            
        if row_size%2 ==0: #for rows with even key numbers
            for row_num in range(1,row_size+1, 2):
                sp.append((config.rowSpacing*row_num+x_trans-0.5*cc.rowSpacing,
                       config.columnSpacing*col_num-0.5*cc.columnSpacing))

        else: # for rows with odd key numbers
            for row_num in range(0,row_size, 2):
                sp.append((config.rowSpacing*row_num+x_trans-0.0*cc.rowSpacing,
                       config.columnSpacing*col_num-0.5*cc.columnSpacing))
        
        
    return sp

hole_psns= get_screw_positions(cc)
plate.faces(">Z").workplane().pushPoints(hole_psns).hole(cc.screwHoleDiamater)

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [43]:
hole_psns.values()

dict_values([(28.5, 0), (47.5, 0), (66.5, 0), (85.5, 0), (104.5, 0), (123.5, 0), (142.5, 0), (161.5, 0), (0, 19), (19, 19), (38, 19), (57, 19), (76, 19), (95, 19), (114, 19), (133, 19), (152, 19), (171, 19), (190, 19), (0, 38), (19, 38), (38, 38), (57, 38), (76, 38), (95, 38), (114, 38), (133, 38), (152, 38), (171, 38), (190, 38), (0, 57), (19, 57), (38, 57), (57, 57), (76, 57), (95, 57), (114, 57), (133, 57), (152, 57), (171, 57), (190, 57)])

In [59]:
cc.row_key_num

[10, 10, 10, 7, 10]

In [50]:
xx=[1,2,3]

In [65]:
list(reversed(cc.row_key_num)).append(cc.row_key_num[0])

In [69]:
list(reversed( [cc.row_key_num[0]]+ cc.row_key_num)) 

[6, 7, 7, 7, 7]

In [70]:
cc.row_key_num

[7, 7, 7, 6]

In [87]:
5 %2

1